In [1]:
import os
import itertools
import numpy as np
import pandas as pd

In [5]:
dataset_path = '../oxpets_dataset'
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_OXPETS'
lr_0s = np.logspace(-1, -4, num=4)
ns = [37, 370, 3441]
prior_path = '/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior'
random_states = [1001, 2001, 3001]
prior_type = 'nonlearned'
wandb_project = 'tuned_OXPETS'
weight_decays = np.append(np.logspace(-2, -6, num=5), 0)
count = -1
for lr_0, n, random_state, weight_decay in itertools.product(lr_0s, ns, random_states, weight_decays):
    count += 1
    model_name = '{}_lr_0={}_n={}_random_state={}_weight_decay={}'\
    .format(prior_type, lr_0, n, random_state, weight_decay)
    print('    "python ../src/OXPETS_main.py --dataset_path=\'{}\' --experiments_path=\'{}\' --lr_0={} --model_name=\'{}\' --n={} --prior_path=\'{}\' --prior_type=\'{}\' --random_state={} --tune --wandb --wandb_project=\'{}\' --weight_decay={}"'\
          .format(dataset_path, experiments_path, lr_0, model_name, n, prior_path, prior_type, random_state, wandb_project, weight_decay))
print(count)

    "python ../src/OXPETS_main.py --dataset_path='../oxpets_dataset' --experiments_path='/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_OXPETS' --lr_0=0.1 --model_name='nonlearned_lr_0=0.1_n=37_random_state=1001_weight_decay=0.01' --n=37 --prior_path='/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior' --prior_type='nonlearned' --random_state=1001 --tune --wandb --wandb_project='tuned_OXPETS' --weight_decay=0.01"
    "python ../src/OXPETS_main.py --dataset_path='../oxpets_dataset' --experiments_path='/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_OXPETS' --lr_0=0.1 --model_name='nonlearned_lr_0=0.1_n=37_random_state=1001_weight_decay=0.001' --n=37 --prior_path='/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior' --prior_type='nonlearned' --random_state=1001 --tune --wandb --wandb_project='tuned_OXPETS' --weight_decay=0.001"
    "python ../src/OXPETS_main.py --dataset_path='../oxpets_dataset' --experiments_path='/cluster/tufts/hu

In [ ]:
# TODO: Adapated

In [ ]:
# TODO: Learned

In [3]:
def get_df(path):
    df = pd.read_csv(path, index_col='Unnamed: 0')
    return df

def get_val_nll(df):
    return df.val_or_test_nll.values[-1]

def get_val_acc(df):
    return df.val_or_test_acc.values[-1]

def get_last_epoch(df):
    return df.iloc[-1]

def get_hyperparameters(experiments_path, lr_0s, ns, prior_scales, prior_type, random_states, weight_decays):
    columns = ['lr_0', 'n', 'prior_scale', 'prior_type', 'random_state', 'val_acc', 'weight_decay']
    df = pd.DataFrame(columns=columns)
    for n, prior_scale, random_state in itertools.product(ns, prior_scales, random_states):
        best_val_nll = np.inf
        best_hyperparameters = None
        for lr_0, weight_decay in itertools.product(lr_0s, weight_decays):
            if prior_scale:
                model_name = '{}_lr_0={}_n={}_prior_scale={}_random_state={}_weight_decay={}'\
                .format(prior_type, lr_0, n, prior_scale, random_state, weight_decay)
            else:
                model_name = '{}_lr_0={}_n={}_random_state={}_weight_decay={}'\
                .format(prior_type, lr_0, n, random_state, weight_decay)
            path =  '{}/{}.csv'.format(experiments_path, model_name)
            val_nll = get_val_nll(get_df(path))
            val_acc = get_val_acc(get_df(path))
            if val_nll < best_val_nll: best_val_nll = val_nll; best_hyperparameters = [lr_0, n, prior_scale, prior_type, random_state, val_acc, weight_decay]
        df.loc[df.shape[0]] = best_hyperparameters
    return df

def get_best_hyperparameters(experiments_path, lr_0s, ns, prior_scales, prior_type, random_states, weight_decays):
    columns = ['lr_0', 'n', 'prior_scale', 'prior_type', 'random_state', 'val_acc', 'weight_decay']
    df = pd.DataFrame(columns=columns)
    for n, random_state in itertools.product(ns, random_states):
        best_val_nll = np.inf
        best_hyperparameters = None
        for lr_0, prior_scale, weight_decay in itertools.product(lr_0s, prior_scales, weight_decays):
            if prior_scale:
                model_name = '{}_lr_0={}_n={}_prior_scale={}_random_state={}_weight_decay={}'\
                .format(prior_type, lr_0, n, prior_scale, random_state, weight_decay)
            else:
                model_name = '{}_lr_0={}_n={}_random_state={}_weight_decay={}'\
                .format(prior_type, lr_0, n, random_state, weight_decay)
            path =  '{}/{}.csv'.format(experiments_path, model_name)
            val_nll = get_val_nll(get_df(path))
            val_acc = get_val_acc(get_df(path))
            if val_nll < best_val_nll: best_val_nll = val_nll; best_hyperparameters = [lr_0, n, prior_scale, prior_type, random_state, val_acc, weight_decay]
        df.loc[df.shape[0]] = best_hyperparameters
    return df

def get_results(df, experiments_path):
    columns = ['n', 'prior_scale', 'prior_type', 'random_state', 'test_acc', 'test_loss', 'test_nll', 
           'test_prior', 'train_acc', 'train_loss', 'train_nll', 'train_prior']
    results = pd.DataFrame(columns=columns)
    for row_index, row in df.iterrows():
        if row.prior_scale:
            model_name = '{}_lr_0={}_n={}_prior_scale={}_random_state={}_weight_decay={}'\
            .format(row.prior_type, row.lr_0, row.n, row.prior_scale, row.random_state, row.weight_decay)
        else:
            model_name = '{}_lr_0={}_n={}_random_state={}_weight_decay={}'\
            .format(row.prior_type, row.lr_0, row.n, row.random_state, row.weight_decay)
        path =  '{}/{}.csv'.format(experiments_path, model_name)
        last_epoch = get_last_epoch(get_df(path))
        results_row = [int(row.n), row.prior_scale, row.prior_type, int(row.random_state), 
                       last_epoch.val_or_test_acc, last_epoch.val_or_test_loss, 
                       last_epoch.val_or_test_nll, last_epoch.val_or_test_prior, 
                       last_epoch.train_acc, last_epoch.train_loss, 
                       last_epoch.train_nll, last_epoch.train_prior]
        results.loc[results.shape[0]] = results_row
    return results

In [4]:
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_OXPETS'
lr_0s = np.logspace(-1, -4, num=4)
ns = [37, 370, 3441]
prior_scales = [None]
prior_type = 'nonlearned'
random_states = [1001, 2001, 3001]
weight_decays = np.append(np.logspace(-2, -6, num=5), 0)
nonlearned_hyperparameters = get_best_hyperparameters(experiments_path, lr_0s, ns, prior_scales, prior_type, random_states, weight_decays)
nonlearned_hyperparameters

,lr_0,n,prior_scale,prior_type,random_state,val_acc,weight_decay
0,0.0010,37,None,nonlearned,1001,0.135135,0.010000
1,0.0010,37,None,nonlearned,2001,0.135135,0.010000
2,0.0010,37,None,nonlearned,3001,0.162162,0.010000
3,0.0100,370,None,nonlearned,1001,0.689189,0.000010
4,0.0010,370,None,nonlearned,2001,0.635135,0.000001
5,0.0001,370,None,nonlearned,3001,0.554054,0.000000
6,0.0100,3441,None,nonlearned,1001,0.897898,0.001000
7,0.0100,3441,None,nonlearned,2001,0.897898,0.001000
8,0.0100,3441,None,nonlearned,3001,0.911411,0.000001


In [8]:
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_OXPETS'
lr_0s = np.logspace(-1, -4, num=4)
ns = [37, 370, 3441]
prior_scales = [None]
prior_type = 'adapted'
random_states = [1001, 2001, 3001]
weight_decays = np.append(np.logspace(-2, -6, num=5), 0)
adapted_hyperparameters = get_best_hyperparameters(experiments_path, lr_0s, ns, prior_scales, prior_type, random_states, weight_decays)
adapted_hyperparameters

,lr_0,n,prior_scale,prior_type,random_state,val_acc,weight_decay
0,0.0001,37,None,adapted,1001,0.135135,0.000010
1,0.0001,37,None,adapted,2001,0.135135,0.000001
2,0.0001,37,None,adapted,3001,0.162162,0.000001
3,0.0100,370,None,adapted,1001,0.702703,0.000010
4,0.1000,370,None,adapted,2001,0.540541,0.010000
5,0.1000,370,None,adapted,3001,0.675676,0.010000
6,0.0100,3441,None,adapted,1001,0.896396,0.001000
7,0.0100,3441,None,adapted,2001,0.899399,0.001000
8,0.0100,3441,None,adapted,3001,0.920420,0.001000


In [10]:
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/retrained_OXPETS'
nonlearned_results = get_results(nonlearned_hyperparameters, experiments_path)
adapted_results = get_results(adapted_hyperparameters, experiments_path)
results = pd.concat([nonlearned_results, adapted_results])
results

,n,prior_scale,prior_type,random_state,test_acc,test_loss,test_nll,test_prior,train_acc,train_loss,train_nll,train_prior
0,37,None,nonlearned,1001,0.172697,3.327513,3.327513,0.000000,1.000000,0.003454,0.003454,0.000000
1,37,None,nonlearned,2001,0.145260,3.398588,3.398588,0.000000,1.000000,0.003515,0.003515,0.000000
2,37,None,nonlearned,3001,0.200102,3.250308,3.250308,0.000000,1.000000,0.003434,0.003434,0.000000
3,370,None,nonlearned,1001,0.575321,1.720191,1.720191,0.000000,1.000000,0.000037,0.000037,0.000000
4,370,None,nonlearned,2001,0.537177,1.828269,1.828269,0.000000,1.000000,0.000633,0.000633,0.000000
5,370,None,nonlearned,3001,0.532070,1.782722,1.782722,0.000000,0.991892,0.156752,0.156752,0.000000
6,3441,None,nonlearned,1001,0.864874,0.555606,0.555606,0.000000,1.000000,0.000219,0.000219,0.000000
7,3441,None,nonlearned,2001,0.866187,0.547540,0.547540,0.000000,1.000000,0.000212,0.000212,0.000000
8,3441,None,nonlearned,3001,0.859838,0.655502,0.655502,0.000000,1.000000,0.000021,0.000021,0.000000
0,37,None,adapted,1001,0.181212,3.276047,3.276021,0.000026,1.000000,0.014836,0.014810,0.000026


In [11]:
grouped_results = results.groupby(['n', 'prior_type']).agg(lambda x: tuple(x))
columns = ['test_acc', 'train_acc']
for column in columns:
    grouped_results['{}_mean'.format(column)] = grouped_results[column].apply(lambda item: np.mean(item))
    grouped_results['{}_std'.format(column)] = grouped_results[column].apply(lambda item: np.std(item))
    grouped_results['{}_min'.format(column)] = grouped_results[column].apply(lambda item: np.min(item))
    grouped_results['{}_max'.format(column)] = grouped_results[column].apply(lambda item: np.max(item))
grouped_results = grouped_results.reset_index()
grouped_results

,n,prior_type,prior_scale,random_state,test_acc,test_loss,test_nll,test_prior,train_acc,train_loss,train_nll,train_prior,test_acc_mean,test_acc_std,test_acc_min,test_acc_max,train_acc_mean,train_acc_std,train_acc_min,train_acc_max
0,37,adapted,"(None, None, None)","(1001, 2001, 3001)","(0.181212097406, 0.151470527053, 0.207152098417)","(3.27604684293, 3.34188798319, 3.18484062653)","(3.2760208553, 3.34188536059, 3.18483800945)","(2.59571570496e-05, 2.53887264989e-06, 2.58759...","(1.0, 1.0, 1.0)","(0.0148364333436, 0.0149347297847, 0.015300299...","(0.0148104764521, 0.0149321909994, 0.015297712...","(2.59571570496e-05, 2.53887264989e-06, 2.58759...",0.179945,0.022750,0.151471,0.207152,1.000000,0.000000,1.000000,1.0
1,37,nonlearned,"(None, None, None)","(1001, 2001, 3001)","(0.17269654572, 0.145260065794, 0.200102090836)","(3.32751329071, 3.39858794323, 3.25030802127)","(3.32751329071, 3.39858794323, 3.25030802127)","(0.0, 0.0, 0.0)","(1.0, 1.0, 1.0)","(0.00345449871384, 0.00351492338814, 0.0034335...","(0.00345449871384, 0.00351492338814, 0.0034335...","(0.0, 0.0, 0.0)",0.172686,0.022389,0.145260,0.200102,1.000000,0.000000,1.000000,1.0
2,370,adapted,"(None, None, None)","(1001, 2001, 3001)","(0.573259949684, 0.554724514484, 0.567416131496)","(1.72257755959, 1.67972723292, 1.80601390482)","(1.72246288025, 1.67389104691, 1.80052900535)","(0.000114704402222, 0.00583617715165, 0.005484...","(1.0, 1.0, 1.0)","(0.000151085477351, 0.00663459922464, 0.006123...","(3.63810751288e-05, 0.000798421896163, 0.00063...","(0.000114704402222, 0.00583617715165, 0.005484...",0.565134,0.007737,0.554725,0.573260,1.000000,0.000000,1.000000,1.0
3,370,nonlearned,"(None, None, None)","(1001, 2001, 3001)","(0.575320959091, 0.537177324295, 0.532069563866)","(1.72019144146, 1.82826946762, 1.78272174171)","(1.72019144146, 1.82826946762, 1.78272174171)","(0.0, 0.0, 0.0)","(1.0, 1.0, 0.991891860962)","(3.67180605158e-05, 0.0006326181232, 0.1567517...","(3.67180605158e-05, 0.0006326181232, 0.1567517...","(0.0, 0.0, 0.0)",0.548189,0.019298,0.532070,0.575321,0.997297,0.003822,0.991892,1.0
4,3441,adapted,"(None, None, None)","(1001, 2001, 3001)","(0.872254252434, 0.863352477551, 0.864451229572)","(0.566249709732, 0.583710750293, 0.596397891165)","(0.534827123183, 0.554832540398, 0.568092595267)","(0.0314225852489, 0.0288781989366, 0.028305305...","(1.0, 1.0, 1.0)","(0.0315601872819, 0.0290197273102, 0.028443414...","(0.00013760190071, 0.000141528476665, 0.000138...","(0.0314225852489, 0.0288781989366, 0.028305305...",0.866686,0.003963,0.863352,0.872254,1.000000,0.000000,1.000000,1.0
5,3441,nonlearned,"(None, None, None)","(1001, 2001, 3001)","(0.86487364769, 0.866187036037, 0.859837651253)","(0.555605979366, 0.547539691277, 0.655501988781)","(0.555605979366, 0.547539691277, 0.655501988781)","(0.0, 0.0, 0.0)","(1.0, 1.0, 1.0)","(0.000218813053151, 0.000211618911242, 2.14034...","(0.000218813053151, 0.000211618911242, 2.14034...","(0.0, 0.0, 0.0)",0.863633,0.002737,0.859838,0.866187,1.000000,0.000000,1.000000,1.0


In [12]:
grouped_results[['n', 'prior_type', 'test_acc_mean', 'test_acc_min', 'test_acc_max']]

,n,prior_type,test_acc_mean,test_acc_min,test_acc_max
0,37,adapted,0.179945,0.151471,0.207152
1,37,nonlearned,0.172686,0.145260,0.200102
2,370,adapted,0.565134,0.554725,0.573260
3,370,nonlearned,0.548189,0.532070,0.575321
4,3441,adapted,0.866686,0.863352,0.872254
5,3441,nonlearned,0.863633,0.859838,0.866187
